In [1]:
import os
import cv2
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define paths and target names
data_dir_train = "/kaggle/input/fyp-dataset/RAFDB/RAFDB/train/"
data_dir_test = "/kaggle/input/fyp-dataset/RAFDB/RAFDB/test/"
target_names = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
NUM_CLASSES = len(target_names)

2024-03-19 15:59:40.335203: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-19 15:59:40.335479: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-19 15:59:40.528580: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Define input size
input_shape = (48, 48, 1)

In [18]:
# Function to load data
def load_data(data_dir, target_names, input_shape):
    img_arr = []
    img_label = []
    label_to_text = {}
    label = 0

    for dir_ in os.listdir(data_dir):
        if dir_ in target_names:
            for f in os.listdir(data_dir + dir_ + "/"):
                img = cv2.imread(data_dir + dir_ + "/" + f)
                img = cv2.resize(img, input_shape[:2])  # Resize the image

                # Convert image to grayscale if it's RGB
                if img.shape[-1] == 3:
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

                img_arr.append(img)
                img_label.append(label)
            print(f"loaded {dir_} images to numpy arrays...")
            label_to_text[label] = dir_
            label += 1

    img_arr = np.array(img_arr)
    img_label = np.array(img_label)
    img_label = OneHotEncoder().fit_transform(img_label.reshape(-1, 1)).toarray()  # Convert to dense array

    return img_arr, img_label, label_to_text

# Define input shape
input_shape = (48, 48, 1) # Update input shape based on loaded image size

# Load training data
img_arr_train, img_label_train, label_to_text_train = load_data(data_dir_train, target_names, input_shape)

# Load test data
img_arr_test, img_label_test, label_to_text_test = load_data(data_dir_test, target_names, input_shape)

# Check shapes and labels
print("Training data shapes:", img_arr_train.shape, img_label_train.shape, label_to_text_train)
print("Test data shapes:", img_arr_test.shape, img_label_test.shape, label_to_text_test)

loaded surprise images to numpy arrays...
loaded fear images to numpy arrays...
loaded angry images to numpy arrays...
loaded neutral images to numpy arrays...
loaded sad images to numpy arrays...
loaded disgust images to numpy arrays...
loaded happy images to numpy arrays...
loaded surprise images to numpy arrays...
loaded fear images to numpy arrays...
loaded angry images to numpy arrays...
loaded neutral images to numpy arrays...
loaded sad images to numpy arrays...
loaded disgust images to numpy arrays...
loaded happy images to numpy arrays...
Training data shapes: (12271, 48, 48) (12271, 7) {0: 'surprise', 1: 'fear', 2: 'angry', 3: 'neutral', 4: 'sad', 5: 'disgust', 6: 'happy'}
Test data shapes: (3068, 48, 48) (3068, 7) {0: 'surprise', 1: 'fear', 2: 'angry', 3: 'neutral', 4: 'sad', 5: 'disgust', 6: 'happy'}


In [19]:
def create_cnn_model(input_shape, NUM_CLASSES):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(NUM_CLASSES, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [20]:
# Train and evaluate the model
print(f"Training model with input shape: {input_shape}")
model = create_cnn_model(input_shape, NUM_CLASSES)

# Preprocess the data (normalize pixel values)
#img_arr_train = img_arr_train / 255.
#img_arr_test = img_arr_test / 255.

# Train the model
model.fit(img_arr_train, img_label_train, epochs=10, validation_data=(img_arr_test, img_label_test))

# Evaluate the model on test data
loss, accuracy = model.evaluate(img_arr_test, img_label_test)
print(f"Test loss: {loss}, Test accuracy: {accuracy}")

Training model with input shape: (48, 48, 1)


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/10
384/384 ━━━━━━━━━━━━━━━━━━━━ 19s 45ms/step - accuracy: 0.4192 - loss: 6.3354 - val_accuracy: 0.6033 - val_loss: 1.0983
Epoch 2/10
384/384 ━━━━━━━━━━━━━━━━━━━━ 20s 43ms/step - accuracy: 0.6268 - loss: 1.0828 - val_accuracy: 0.6486 - val_loss: 0.9916
Epoch 3/10
384/384 ━━━━━━━━━━━━━━━━━━━━ 20s 42ms/step - accuracy: 0.6690 - loss: 0.9242 - val_accuracy: 0.6516 - val_loss: 1.0132
Epoch 4/10
384/384 ━━━━━━━━━━━━━━━━━━━━ 17s 43ms/step - accuracy: 0.7154 - loss: 0.8157 - val_accuracy: 0.6545 - val_loss: 1.0105
Epoch 5/10
384/384 ━━━━━━━━━━━━━━━━━━━━ 16s 42ms/step - accuracy: 0.7452 - loss: 0.7257 - val_accuracy: 0.6467 - val_loss: 1.0607
Epoch 6/10
384/384 ━━━━━━━━━━━━━━━━━━━━ 17s 43ms/step - accuracy: 0.7800 - loss: 0.6345 - val_accuracy: 0.6767 - val_loss: 1.0586
Epoch 7/10
384/384 ━━━━━━━━━━━━━━━━━━━━ 16s 42ms/step - accuracy: 0.8057 - loss: 0.5469 - val_accuracy: 0.6467 - val_loss: 1.2522
Epoch 8/10
384/384 ━━━━━━━━━━━━━━━━━━━━ 21s 43ms/step - accuracy: 0.8248 - loss: 0.4990 - 

# Define input size
input_shape = (64, 64, 1)

In [9]:
# Function to load data
def load_data(data_dir, target_names, input_shape):
    img_arr = []
    img_label = []
    label_to_text = {}
    label = 0

    for dir_ in os.listdir(data_dir):
        if dir_ in target_names:
            for f in os.listdir(data_dir + dir_ + "/"):
                img = cv2.imread(data_dir + dir_ + "/" + f)
                img = cv2.resize(img, input_shape[:2])  # Resize the image

                # Convert image to grayscale if it's RGB
                if img.shape[-1] == 3:
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

                img_arr.append(img)
                img_label.append(label)
            print(f"loaded {dir_} images to numpy arrays...")
            label_to_text[label] = dir_
            label += 1

    img_arr = np.array(img_arr)
    img_label = np.array(img_label)
    img_label = OneHotEncoder().fit_transform(img_label.reshape(-1, 1)).toarray()  # Convert to dense array

    return img_arr, img_label, label_to_text

# Define input shape
input_shape = (64, 64, 1) # Update input shape based on loaded image size

# Load training data
img_arr_train, img_label_train, label_to_text_train = load_data(data_dir_train, target_names, input_shape)

# Load test data
img_arr_test, img_label_test, label_to_text_test = load_data(data_dir_test, target_names, input_shape)

# Check shapes and labels
print("Training data shapes:", img_arr_train.shape, img_label_train.shape, label_to_text_train)
print("Test data shapes:", img_arr_test.shape, img_label_test.shape, label_to_text_test)

loaded surprise images to numpy arrays...
loaded fear images to numpy arrays...
loaded angry images to numpy arrays...
loaded neutral images to numpy arrays...
loaded sad images to numpy arrays...
loaded disgust images to numpy arrays...
loaded happy images to numpy arrays...
loaded surprise images to numpy arrays...
loaded fear images to numpy arrays...
loaded angry images to numpy arrays...
loaded neutral images to numpy arrays...
loaded sad images to numpy arrays...
loaded disgust images to numpy arrays...
loaded happy images to numpy arrays...
Training data shapes: (12271, 64, 64) (12271, 7) {0: 'surprise', 1: 'fear', 2: 'angry', 3: 'neutral', 4: 'sad', 5: 'disgust', 6: 'happy'}
Test data shapes: (3068, 64, 64) (3068, 7) {0: 'surprise', 1: 'fear', 2: 'angry', 3: 'neutral', 4: 'sad', 5: 'disgust', 6: 'happy'}


In [10]:
def create_cnn_model(input_shape, NUM_CLASSES):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(NUM_CLASSES, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [11]:
# Train and evaluate the model
print(f"Training model with input shape: {input_shape}")
model = create_cnn_model(input_shape, NUM_CLASSES)

# Train the model
model.fit(img_arr_train, img_label_train, epochs=10, validation_data=(img_arr_test, img_label_test))

# Evaluate the model on test data
loss, accuracy = model.evaluate(img_arr_test, img_label_test)
print(f"Test loss: {loss}, Test accuracy: {accuracy}")

Training model with input shape: (64, 64, 1)


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/10
384/384 ━━━━━━━━━━━━━━━━━━━━ 33s 82ms/step - accuracy: 0.4441 - loss: 7.9696 - val_accuracy: 0.6141 - val_loss: 1.0936
Epoch 2/10
384/384 ━━━━━━━━━━━━━━━━━━━━ 32s 82ms/step - accuracy: 0.6582 - loss: 0.9846 - val_accuracy: 0.6610 - val_loss: 0.9675
Epoch 3/10
384/384 ━━━━━━━━━━━━━━━━━━━━ 32s 82ms/step - accuracy: 0.7182 - loss: 0.7941 - val_accuracy: 0.6675 - val_loss: 1.0041
Epoch 4/10
384/384 ━━━━━━━━━━━━━━━━━━━━ 31s 80ms/step - accuracy: 0.7764 - loss: 0.6405 - val_accuracy: 0.6780 - val_loss: 0.9868
Epoch 5/10
384/384 ━━━━━━━━━━━━━━━━━━━━ 41s 80ms/step - accuracy: 0.8276 - loss: 0.4997 - val_accuracy: 0.6662 - val_loss: 1.1100
Epoch 6/10
384/384 ━━━━━━━━━━━━━━━━━━━━ 41s 80ms/step - accuracy: 0.8706 - loss: 0.3779 - val_accuracy: 0.6617 - val_loss: 1.2400
Epoch 7/10
384/384 ━━━━━━━━━━━━━━━━━━━━ 30s 79ms/step - accuracy: 0.8988 - loss: 0.2941 - val_accuracy: 0.6519 - val_loss: 1.4204
Epoch 8/10
384/384 ━━━━━━━━━━━━━━━━━━━━ 41s 80ms/step - accuracy: 0.9202 - loss: 0.2328 - 

# Define input size
input_shape = (128, 128, 1)

In [12]:
# Function to load data
def load_data(data_dir, target_names, input_shape):
    img_arr = []
    img_label = []
    label_to_text = {}
    label = 0

    for dir_ in os.listdir(data_dir):
        if dir_ in target_names:
            for f in os.listdir(data_dir + dir_ + "/"):
                img = cv2.imread(data_dir + dir_ + "/" + f)
                img = cv2.resize(img, input_shape[:2])  # Resize the image

                # Convert image to grayscale if it's RGB
                if img.shape[-1] == 3:
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

                img_arr.append(img)
                img_label.append(label)
            print(f"loaded {dir_} images to numpy arrays...")
            label_to_text[label] = dir_
            label += 1

    img_arr = np.array(img_arr)
    img_label = np.array(img_label)
    img_label = OneHotEncoder().fit_transform(img_label.reshape(-1, 1)).toarray()  # Convert to dense array

    return img_arr, img_label, label_to_text

# Define input shape
input_shape = (128, 128, 1) # Update input shape based on loaded image size

# Load training data
img_arr_train, img_label_train, label_to_text_train = load_data(data_dir_train, target_names, input_shape)

# Load test data
img_arr_test, img_label_test, label_to_text_test = load_data(data_dir_test, target_names, input_shape)

# Check shapes and labels
print("Training data shapes:", img_arr_train.shape, img_label_train.shape, label_to_text_train)
print("Test data shapes:", img_arr_test.shape, img_label_test.shape, label_to_text_test)

loaded surprise images to numpy arrays...
loaded fear images to numpy arrays...
loaded angry images to numpy arrays...
loaded neutral images to numpy arrays...
loaded sad images to numpy arrays...
loaded disgust images to numpy arrays...
loaded happy images to numpy arrays...
loaded surprise images to numpy arrays...
loaded fear images to numpy arrays...
loaded angry images to numpy arrays...
loaded neutral images to numpy arrays...
loaded sad images to numpy arrays...
loaded disgust images to numpy arrays...
loaded happy images to numpy arrays...
Training data shapes: (12271, 128, 128) (12271, 7) {0: 'surprise', 1: 'fear', 2: 'angry', 3: 'neutral', 4: 'sad', 5: 'disgust', 6: 'happy'}
Test data shapes: (3068, 128, 128) (3068, 7) {0: 'surprise', 1: 'fear', 2: 'angry', 3: 'neutral', 4: 'sad', 5: 'disgust', 6: 'happy'}


In [13]:
def create_cnn_model(input_shape, NUM_CLASSES):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(NUM_CLASSES, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [14]:
# Train and evaluate the model
print(f"Training model with input shape: {input_shape}")
model = create_cnn_model(input_shape, NUM_CLASSES)

# Train the model
model.fit(img_arr_train, img_label_train, epochs=10, validation_data=(img_arr_test, img_label_test))

# Evaluate the model on test data
loss, accuracy = model.evaluate(img_arr_test, img_label_test)
print(f"Test loss: {loss}, Test accuracy: {accuracy}")

Training model with input shape: (128, 128, 1)


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/10
384/384 ━━━━━━━━━━━━━━━━━━━━ 143s 368ms/step - accuracy: 0.4448 - loss: 22.1439 - val_accuracy: 0.6053 - val_loss: 1.1069
Epoch 2/10
384/384 ━━━━━━━━━━━━━━━━━━━━ 141s 365ms/step - accuracy: 0.6664 - loss: 0.9537 - val_accuracy: 0.6232 - val_loss: 1.0631
Epoch 3/10
384/384 ━━━━━━━━━━━━━━━━━━━━ 142s 367ms/step - accuracy: 0.7351 - loss: 0.7559 - val_accuracy: 0.6620 - val_loss: 1.0251
Epoch 4/10
384/384 ━━━━━━━━━━━━━━━━━━━━ 138s 357ms/step - accuracy: 0.8073 - loss: 0.5516 - val_accuracy: 0.6369 - val_loss: 1.2296
Epoch 5/10
384/384 ━━━━━━━━━━━━━━━━━━━━ 146s 367ms/step - accuracy: 0.8527 - loss: 0.4227 - val_accuracy: 0.6454 - val_loss: 1.3231
Epoch 6/10
384/384 ━━━━━━━━━━━━━━━━━━━━ 141s 364ms/step - accuracy: 0.9046 - loss: 0.2821 - val_accuracy: 0.6258 - val_loss: 1.5850
Epoch 7/10
384/384 ━━━━━━━━━━━━━━━━━━━━ 139s 357ms/step - accuracy: 0.9279 - loss: 0.2407 - val_accuracy: 0.6323 - val_loss: 1.8809
Epoch 8/10
384/384 ━━━━━━━━━━━━━━━━━━━━ 141s 368ms/step - accuracy: 0.9560 

# Define input size
input_shape = (256, 256, 1)

In [15]:
# Function to load data
def load_data(data_dir, target_names, input_shape):
    img_arr = []
    img_label = []
    label_to_text = {}
    label = 0

    for dir_ in os.listdir(data_dir):
        if dir_ in target_names:
            for f in os.listdir(data_dir + dir_ + "/"):
                img = cv2.imread(data_dir + dir_ + "/" + f)
                img = cv2.resize(img, input_shape[:2])  # Resize the image

                # Convert image to grayscale if it's RGB
                if img.shape[-1] == 3:
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

                img_arr.append(img)
                img_label.append(label)
            print(f"loaded {dir_} images to numpy arrays...")
            label_to_text[label] = dir_
            label += 1

    img_arr = np.array(img_arr)
    img_label = np.array(img_label)
    img_label = OneHotEncoder().fit_transform(img_label.reshape(-1, 1)).toarray()  # Convert to dense array

    return img_arr, img_label, label_to_text

# Define input shape
input_shape = (256, 256, 1) # Update input shape based on loaded image size

# Load training data
img_arr_train, img_label_train, label_to_text_train = load_data(data_dir_train, target_names, input_shape)

# Load test data
img_arr_test, img_label_test, label_to_text_test = load_data(data_dir_test, target_names, input_shape)

# Check shapes and labels
print("Training data shapes:", img_arr_train.shape, img_label_train.shape, label_to_text_train)
print("Test data shapes:", img_arr_test.shape, img_label_test.shape, label_to_text_test)

loaded surprise images to numpy arrays...
loaded fear images to numpy arrays...
loaded angry images to numpy arrays...
loaded neutral images to numpy arrays...
loaded sad images to numpy arrays...
loaded disgust images to numpy arrays...
loaded happy images to numpy arrays...
loaded surprise images to numpy arrays...
loaded fear images to numpy arrays...
loaded angry images to numpy arrays...
loaded neutral images to numpy arrays...
loaded sad images to numpy arrays...
loaded disgust images to numpy arrays...
loaded happy images to numpy arrays...
Training data shapes: (12271, 256, 256) (12271, 7) {0: 'surprise', 1: 'fear', 2: 'angry', 3: 'neutral', 4: 'sad', 5: 'disgust', 6: 'happy'}
Test data shapes: (3068, 256, 256) (3068, 7) {0: 'surprise', 1: 'fear', 2: 'angry', 3: 'neutral', 4: 'sad', 5: 'disgust', 6: 'happy'}


In [16]:
def create_cnn_model(input_shape, NUM_CLASSES):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(NUM_CLASSES, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [17]:
# Train and evaluate the model
print(f"Training model with input shape: {input_shape}")
model = create_cnn_model(input_shape, NUM_CLASSES)

# Train the model
model.fit(img_arr_train, img_label_train, epochs=10, validation_data=(img_arr_test, img_label_test))

# Evaluate the model on test data
loss, accuracy = model.evaluate(img_arr_test, img_label_test)
print(f"Test loss: {loss}, Test accuracy: {accuracy}")

Training model with input shape: (256, 256, 1)


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/10
384/384 ━━━━━━━━━━━━━━━━━━━━ 678s 2s/step - accuracy: 0.3926 - loss: 93.6683 - val_accuracy: 0.3898 - val_loss: 1.6500
Epoch 2/10
384/384 ━━━━━━━━━━━━━━━━━━━━ 681s 2s/step - accuracy: 0.4316 - loss: 1.5699 - val_accuracy: 0.4404 - val_loss: 1.5261
Epoch 3/10
384/384 ━━━━━━━━━━━━━━━━━━━━ 682s 2s/step - accuracy: 0.5471 - loss: 1.2614 - val_accuracy: 0.5023 - val_loss: 1.5581
Epoch 4/10
384/384 ━━━━━━━━━━━━━━━━━━━━ 682s 2s/step - accuracy: 0.6699 - loss: 0.9281 - val_accuracy: 0.4889 - val_loss: 1.6823
Epoch 5/10
384/384 ━━━━━━━━━━━━━━━━━━━━ 687s 2s/step - accuracy: 0.7388 - loss: 0.7610 - val_accuracy: 0.5189 - val_loss: 2.1434
Epoch 6/10
384/384 ━━━━━━━━━━━━━━━━━━━━ 682s 2s/step - accuracy: 0.8033 - loss: 0.5700 - val_accuracy: 0.5267 - val_loss: 2.4872
Epoch 7/10
384/384 ━━━━━━━━━━━━━━━━━━━━ 679s 2s/step - accuracy: 0.8600 - loss: 0.4142 - val_accuracy: 0.5271 - val_loss: 2.7262
Epoch 8/10
384/384 ━━━━━━━━━━━━━━━━━━━━ 679s 2s/step - accuracy: 0.9014 - loss: 0.3030 - val_acc